In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Introduction**

Welcome to this notebook. The idea of this jumble of code is to explore how to approach this problem without too much technical knowledge. Over the previous year I learned a lot about graph theory and reinforcement learning, so I am using this competition to test some concepts.

As a start the issue at hand can be phrased as a graph problem. Chaining the different permutations of the symbols is equivalent of following a route through this graph, the nodes or vertices being the permutations. What is interesting is that this graph is fully connected (you can go directly from each node a to node b), the graph is weighted and directed (the distance from node a to node b is not the same as the distance from node b to node a). This is reflected in the form of the distance matrix, also known as adjacency matrix, which is dense, (almost) no zeros and asymmetric.

On this graph, you have three agents that try to cover all of it as efficiently as possible, with a subset of the nodes that need to be covered by all of them first. This splits the problem in two parts:

1. Go through the first 120 nodes (permutations with fixed first two entries, 120 = (7 - 2)! = 5!). This is a path with a fixed length (120 * 7 = 840) and if I am correct it is not possible to shorten it. However, what is possible is to choose paths that go through more than 120 nodes. Think of this as in 2d three nodes being on a straight line connecting them, if you from node one to three you have to go through two. This allows to cover more nodes that 120 in the first part of the route. Also, the three agents do not need to take the same route/order for these first 120 nodes. In fact, you want them to be as different as possible to cover more nodes. This can be realized for example by the first agent going through the nodes in order, the second agent in reverse order and the third agent going first through the even and then the odd nodes. (The nodes always have some id on which you can sort them). We optimize this by randomly permutating two elements of a random list to cover more nodes with the starting three routes.

2. After having found the first part of the three routes, we greedily add the remaining nodes to the three routes. This means that we try to find for each route the next node with the shortest distance to the last node of each route and then add the node which will lead to the minimum increase in max length of all routes. As the selection of the next node is not necessarily unique, we break ties by choosing randomly from the eligible candidates.

# Imports

In [ ]:
import tqdm
import random
from collections import deque
from random import randint, sample, choice

# Loading Data

****

In [ ]:
permutations = pd.read_csv("/kaggle/input/santa-2021/permutations.csv")
distances = pd.read_csv("/kaggle/input/santa-2021/distance_matrix.csv")
permutations_ = [p[0] for p in permutations.values].copy()


In [ ]:
permutations.head()

# Distance Formatting
We format the distance matrix as a map that maps each permutations to a map of distances, for each distance a list of all the permutations with that distance. We also prepare maps that connect id and permutation string. We will use the id further as nodes for our graph.

In [ ]:
distance_dict = {}

for i, row in distances.iterrows():
    row_index = row.values[0]
    
    vals = row[1:]
    
    ldict = {}
    for j in range(1,8):
        ldict[j] = vals[vals == j].index.values
    
    distance_dict[row_index] = ldict
    
d = np.array(distances.values[:,1:], np.int32)

In [ ]:
ids = {s[0]:i for i, s in enumerate(permutations.values)}

distance_dict_id = {}

for k, v in tqdm.tqdm(distance_dict.items()):
    local = {}
    for k_, v_ in v.items():
        local[k_] = [ids[c] for c in v_]
    distance_dict_id[ids[k]] = local

In [ ]:
print(row_index)
distance_dict[row_index][2]

In [ ]:
list(distance_dict_id.keys())[0]

# Score function
Calculation of distance for the score, i.e. distance of each route in the list and taking maximum of those.

In [ ]:
def get_score_graph(list_of_lists):
    score = 0
    
    for l in list_of_lists:
        total = 7
        for i in range(1,len(l)):
            total += d[l[i - 1], l[i]]
        score = max(score, total)
    return score


# First part 120 (or more)

We first build the three basic routes that cover the first 120 permutations, but we want them to actually cover as many permutations as possible. For this we first make a set of all nodes/permutations.

In [ ]:
nodes_covered = {}

for id1 in range(120):
    nodes_covered_ = {}
    perm1 = permutations_[id1]
    for id2 in range(120):
        n_ = []
        perm2 = permutations_[id2]
        for dist in range(0,8):
            new_string = perm1[dist:] + perm2[:dist]
            if new_string in permutations_:
                n_.append(ids[new_string])
        nodes_covered_ [id2] = n_
    nodes_covered[id1] = nodes_covered_
        
nodes_covered[0][1]

In [ ]:
best_routs = []
best_seen = 0

route_lst = list(range(0,120))

for k in tqdm.trange(500):
    routs = []

    seen_nodes = set(route_lst)
    
    routs = [[i] for i in random.sample(route_lst, 3)]
    nodes_to_cover = [set(route_lst) for _ in range(3)]
    
    for j, r in enumerate(routs):
        nodes_to_cover[j].remove(r[0])
    
    while nodes_to_cover[0] or nodes_to_cover[1] or nodes_to_cover[2]:
        for j, route in enumerate(routs):
            nodes_covered_ = nodes_covered[route[-1]]

            best_next = None
            best_value = -1
            
            l_ = lambda n_ : n_ not in seen_nodes

            for n in nodes_to_cover[j]:
                val = sum(map(l_, nodes_covered_[n])) + 0.01 * random.random()

                if val > best_value:
                    best_next = n
                    best_value = val
            
            nodes_to_cover[j].remove(best_next)
            route.append(best_next)

            seen_nodes.update(nodes_covered_[best_next])

    if len(seen_nodes) > best_seen:
        best_routs = routs
        best_seen = len(seen_nodes)
        
#         if best_seen == 840:
#             break
        
print(best_seen)

In [ ]:
list_of_list = [[permutations_[i] for i in l] for l in routs]

for l in list_of_list:
    seen_nodes = set()
    seen_nodes.update(nodes_covered)
    for i in range(1,120):
        seen_nodes.update(nodes_covered[ids[l[i - 1]]][ids[l[i]]])
        seen_nodes.update(nodes_covered[ids[l[i]]][ids[l[i]]])
    print(len(seen_nodes))

In [ ]:
base_score = len(permutations_[120:])
best_list_of_list = list_of_list.copy()

result = ["".join(l) for l in list_of_list]
result0 = result[0]
result1 = result[1]
result2 = result[2]

l_ = lambda p: (p in result0) or (p in result1) or (p in result2)
last_permutations = permutations_[120:]

min_score = sum(map(l_, last_permutations))


sample_range = list(range(0,120))

loop_size = 5000

random_i1 = np.random.randint(120, size = loop_size)
random_i2 = np.random.randint(120, size = loop_size)
# random_i3 = np.random.randint(120, size = loop_size)
# random_i4 = np.random.randint(120, size = loop_size)
# random_i5 = np.random.randint(120, size = loop_size)

random_l1 = np.random.randint(3, size = loop_size)
import itertools

for k in tqdm.trange(loop_size):
    
    list_of_list = [l.copy() for l in best_list_of_list]

    lst = [random_i1[k], random_i2[k]]
    first = True
    for p in itertools.permutations(lst):
        if first:
            first = False
            continue
        l1 = list_of_list[random_l1[k]]
        l2 = l1.copy()
        
        for i, p_ in zip(lst, p):
            l1[i] = l2[p_]

        result = ["".join(l) for l in list_of_list]
        result0 = result[0]
        result1 = result[1]
        result2 = result[2]

        l_ = lambda p: (p in result0) or (p in result1) or (p in result2)

        score = sum(map(l_, last_permutations)) + 0.01 * random.random()

        if score > min_score:
            min_score = score
            base_result = result.copy()
            best_list_of_list = list_of_list

            print(min_score)




In [ ]:
sub = set(permutations_[120:])

for p in permutations_[120:]:
    if (p in base_result[0]) or (p in base_result[1]) or (p in base_result[2]):
        sub.remove(p)
        
base_sub = set([ids[s] for s in sub])
base_list_of_lists = [[ids[r[7 * i: i * 7 + 7]] for i in range(120)] for r in base_result]

In [ ]:
def find_best_route(depth):
    
    sub = base_sub.copy()

    list_of_lists = [deque(l) for l in base_list_of_lists]
    list_of_distances = np.zeros(3)
    
    while sub:
        min_total_dist = 100000
        shortest = 0
        min_new_dist = 7
        min_next_closest_id = None
        
        last_words = [l[-1] for l in list_of_lists]
        
        for i, l in enumerate(list_of_lists):
            
            next_closest_id, dist = get_next_graph(l[-1], last_words, distance_dict_id, sub, d, depth)
            
            new_dist = list_of_distances[i] + dist * 2
            
            if new_dist < min_total_dist:
                min_total_dist = new_dist
                shortest = i
                min_new_dist = dist
                min_next_closest_id = next_closest_id
                
                if new_dist < min(list_of_distances):
                    break
                                
        if min_next_closest_id:
            if min_next_closest_id in sub:
                sub.remove(min_next_closest_id)
            shortest_list = list_of_lists[shortest]
            shortest_list.append(min_next_closest_id)
            list_of_distances[shortest] += d[shortest_list[-2], shortest_list[-1]]
        
    
    list_of_lists = remove_end(list_of_lists)
    
#     k_opt(list_of_lists, 8)
    
    return list_of_lists, get_score_graph(list_of_lists)

def get_next_graph(last_word_id, last_words, distance_dict_id, sub_id, d, step = 0):
    
    dist = 1
    distances = distance_dict_id[last_word_id]
    next_closest_id = distances[dist][0]

    if (next_closest_id in sub_id):
          
        if step > 0:
            
#             for last_word in last_words:
#                 if last_word != last_word_id:
#                     dist -= 0.1 * d[next_closest_id, last_word]
                
            dist += get_next_graph(next_closest_id, last_words, distance_dict_id, sub_id, d, step - 1)[1]

        return next_closest_id, dist
    
    candidates = []
    l_ = lambda p: p in sub_id
    
    while (dist < 7) and (not candidates):
        candidates.clear()
        dist += 1
        capp = candidates.append
        for c in filter(l_, distances[dist]):
            capp(c)
        
    if candidates and step > 0:
        next_closest_id = None
        min_dist = 8 * step
        
        for c in candidates:
            
            new_dist = dist + get_next_graph(c, last_words, distance_dict_id, sub_id, d, step - 1)[1] + 0.01 * random.random()
            
            min_d = 100
            
            for last_word in last_words:
                if last_word != last_word_id:
                    min_d = min(min_d, d[last_word, c])

            new_dist -= 0.5 * min_d

            if new_dist < min_dist:
                next_closest_id = c
                min_dist = new_dist
                
        dist = min_dist
            
    elif candidates:
        
        next_closest_id = None
        min_dist = 8
        
        for c in candidates:
            
            new_dist = dist + 0.01 * random.random()
            
            min_d = 100
            
            for last_word in last_words:
                if last_word != last_word_id:
                    min_d = min(min_d, d[last_word, c])

            new_dist -= 0.5 * min_d
            
            if new_dist < min_dist:
                next_closest_id = c
                min_dist = new_dist
                
        dist = min_dist
            
    return next_closest_id, dist

def remove_end(best_list_of_lists):
    current_score = get_score_graph(best_list_of_lists)
    new_score = current_score

    current_score += 1

    while current_score > new_score:
        current_score = new_score

        for j in range(3):
            l = list(best_list_of_lists[j])
            last_word = l[-1]
            
            if last_word in l[:-1]:
                best_list_of_lists[j].pop()
            else:
                for l_ in [l_ for k, l_ in enumerate(best_list_of_lists) if k != j]:
                    if last_word in l_:
                        best_list_of_lists[j].pop()
                        break
                        
            
        new_score = get_score_graph(best_list_of_lists)
    return best_list_of_lists
         
def k_opt(best_list_of_lists, length_):

    for l in best_list_of_lists:

        for i in range(120, len(l)- length_):
            lst = [i + j for j in range(length_)]
            old_total = d[l[i - 1], l[i]]
            for i_ in lst:
                old_total += d[l[i_], l[i_ + 1]]

            start_pos = l[i - 1]
            end_pos = l[i + length_]

            if old_total < 2 * length_:
                continue


            for p in itertools.permutations(lst):

                pos = start_pos
                new_total = 0.0

                for p_ in p:
                    new_pos = l[p_]
                    new_total += d[pos, new_pos]
                    if new_total > old_total:
                        break
                    pos = new_pos
                new_total += d[pos, end_pos]    
                if new_total < old_total:

                    l_slice = list(l)[i:i+length_].copy()

                    for j, p_ in enumerate(p):
                        l[i + j ] = l_slice[p_ - i]

In [ ]:
best_list_of_lists = None

for k in tqdm.trange(10_000):

    r = find_best_route(2)
    
    r_ = r
    list_of_lists = r_[0]
    score = r_[1]

    if best_list_of_lists:
        new_result_len = score
        if new_result_len < shortest_result_len:
            shortest_result_len = new_result_len
            best_list_of_lists = [l.copy() for l in list_of_lists]
            print(shortest_result_len)
    else:
        shortest_result_len = score
        best_list_of_lists = [l.copy() for l in list_of_lists]
        print(shortest_result_len)

In [ ]:
get_score_graph(best_list_of_lists)

# Distance Distribution Visualization

In [ ]:
import matplotlib.pyplot as plt

distance_distributions = []

for l in best_list_of_lists:
    dist = list()
    for i in range(len(l) - 1):
        dist.append(d[l[i], l[i + 1]])
    distance_distributions.append(dist)
    print(sum(dist))
    plt.plot(list(dist)[120:])
    plt.show()
    print(dist[-10:])

In [ ]:
pd.DataFrame(distance_distributions).T.groupby(1).count()

In [ ]:
max_index = 0
max_value = 0

for j, l in enumerate(best_list_of_lists):
    total = 0
    for i in range(1, len(l)):
        total += d[l[i - 1], l[i]]
        
    if total > max_value:
        max_value = total
        max_index = j
        
    print(total)

In [ ]:
l = list(best_list_of_lists[2])[120:]
count = 0

for j in range(len(l) - 1):
    word1 = permutations_[l[j]]
    word2 = permutations_[l[j + 1]]
    for i in range(1,7):
        new_perm = word1[i:] + word2[:i] 
        if new_perm in ids:
            count += 1
            
print(count)

In [ ]:
# loop_size = 1_000_000


# randi1 = [np.random.randint(120, len(city_list_of_list[i]) - 2, size = loop_size) for i in range(3)]
# randi2 = [np.random.randint(120, len(city_list_of_list[i]) - 2, size = loop_size) for i in range(3)]
# randm1 = np.random.randint(0, 2, size = loop_size)
# randm2 = np.random.randint(0, 2, size = loop_size)

# d_ = [get_distance(l) for l in city_list_of_list]

# print(d_)

# for k in tqdm.trange(loop_size):
# #         total = 0

#     for m2, l2 in enumerate(city_list_of_list):
#         m1 = d_.index(max(d_))
# #         m2 = l2

#         i1 = randi1[m1][k] #randint(120, len(l) - 2)
#         i2 = randi2[m2][k] #randint(120, len(l) - 2)

#         if (abs(i1 - i2) < 2):
#             continue

#         l1 = city_list_of_list[m1]
#         li1m1 = l1[i1 - 1]
#         li1 = l1[i1]
#         li1p1 = l1[i1 + 1]

# #         l2 = city_list_of_list[m2]
#         li2m1 = l2[i2 - 1]
#         li2 = l2[i2]
#         li2p1 = l2[i2 + 1]

#         l1_diff = d[li1m1, li2] + d[li2, li1p1] - d[li1m1, li1] - d[li1, li1p1 ] 
#         l2_diff = d[li2m1, li1] + d[li1, li2p1] - d[li2m1, li2] - d[li2, li2p1]

#         if l1_diff + l2_diff < 0:
#             l1[i1], l2[i2] = li2, li1
#             break

# get_score_graph(city_list_of_list)

In [ ]:
current_score = get_score_graph(best_list_of_lists)
new_score = current_score

current_score += 1

while current_score > new_score:
    current_score = new_score

    for j in range(3):
        l = list(best_list_of_lists[j])
        last_word = l[-1]
        last_word_dist = d[l[-2], l[-1]]
        
        for i in range(120, len(l) - 1):
            old_dist = d[l[i - 1], l[i]] + last_word_dist
            new_dist = d[l[i - 1], last_word] + d[last_word, l[i]]
            
            if new_dist < old_dist:
                print('insert')
                best_list_of_lists[j] = l[:-1]
                l.insert(i, last_word)
                break
        
    new_score = get_score_graph(best_list_of_lists)

In [ ]:
get_score_graph(best_list_of_lists)

In [ ]:
string_result = []

for l in best_list_of_lists:
    local_result = permutations_[l[0]]
    
    for i in list(l)[1:]:
        local_result += permutations_[i][-d[ids[local_result[-7:]], ids[permutations_[i]]]:]
    string_result.append(local_result)

shortest_result = string_result

for r in shortest_result:
    print(len(r))

In [ ]:
for p in permutations.values:
    word = p[0]
    found = False
    for r in shortest_result:
        if word in r:
            found = True
    
    if not found:
        print("failure " + word)
        break

In [ ]:
# wildcard_char = '🌟'

# max_r = sorted(shortest_result, key = len)[-1]
# max_index = shortest_result.index(max_r)
# print(len(max_r), max_index)

# last_word = max_r[-7:]
# last_word_index = len(max_r) - 7

# for i, r in enumerate(shortest_result):
#     if last_word[:-1] in r and i != max_index:
#         replace_index = r.index(last_word[:-1]) + 6
#         shortest_result[i] = shortest_result[i][:replace_index] + wildcard_char + shortest_result[i][replace_index + 1]
        
#         l = best_list_of_lists[max_index]
        
#         shortest_result[max_index] = shortest_result[max_index][:-d[l[-2], l[-1]]]
#         break
    


In [ ]:
result_df = pd.DataFrame(shortest_result, columns = ["schedule"])

In [ ]:
result_df.to_csv("submission.csv", index=False)